In [3]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

BASE_DIR = Path(".")
ROT_FILE = BASE_DIR / "neighbor_days_rotations.csv"

DATE_START = pd.Timestamp("2025-12-19")
DATE_END   = pd.Timestamp("2025-12-25")

OUT_CSV  = BASE_DIR / "service_plan_by_date.csv"
OUT_JSON = BASE_DIR / "service_plan_by_date.json"
OUT_SUM  = BASE_DIR / "service_plan_summary.csv"

def normalize_col(s: str) -> str:
    s = str(s).strip().lower()
    s = (s.replace("ç","c").replace("ş","s").replace("ğ","g")
           .replace("ı","i").replace("ö","o").replace("ü","u"))
    s = s.replace(" ", "_").replace("-", "_")
    return s

def truthy(x) -> bool:
    if pd.isna(x):
        return False
    if isinstance(x, (int, float)):
        return x != 0
    s = str(x).strip().lower()
    return s in ["1", "true", "t", "yes", "y", "var", "x", "ok", "evet"]

# İngilizce gün -> kısa anahtar
DAY_MAP = {
    "monday": "mon",
    "tuesday": "tue",
    "wednesday": "wed",
    "thursday": "thu",
    "friday": "fri",
    "saturday": "sat",
    "sunday": "sun"
}

TR_WEEKDAYS = {
    "monday": "Pazartesi",
    "tuesday": "Salı",
    "wednesday": "Çarşamba",
    "thursday": "Perşembe",
    "friday": "Cuma",
    "saturday": "Cumartesi",
    "sunday": "Pazar",
}

# --------------------
# 1) Oku + kolonları normalize et
# --------------------
df = pd.read_csv(ROT_FILE, sep=None, engine="python", on_bad_lines="skip")
df.columns = [normalize_col(c).lstrip("\ufeff") for c in df.columns]  # BOM temizle

print("Faz2 | Kolonlar:", df.columns.tolist())
print("Faz2 | Satır sayısı:", len(df))

# beklenen kolonlar (senin çıktına göre)
NEIGH_COL = "mahalle_adi"
FREQ_COL  = "collection_frequency_(truck_type)"
CRANE_COL = "is_crane_used"

# kolon doğrulama
for col in [NEIGH_COL, FREQ_COL, CRANE_COL]:
    if col not in df.columns:
        raise ValueError(f"Beklenen kolon yok: {col}. Kolonlar: {df.columns.tolist()}")

# frequency kolonunu normalize edelim (string)
df[FREQ_COL] = df[FREQ_COL].astype(str).str.strip()

# Bazı satırlarda frequency 0 gibi gelebiliyor; onları hiç toplama yok diye eleyeceğiz
def parse_days_list(freq_str: str):
    s = str(freq_str).strip().lower()
    if s in ["0", "nan", "none", ""]:
        return set()
    # "Monday, Wednesday, Friday" -> {"mon","wed","fri"}
    parts = [p.strip() for p in s.split(",")]
    days = set()
    for p in parts:
        if p.startswith("mon"): days.add("mon")
        elif p.startswith("tue"): days.add("tue")
        elif p.startswith("wed"): days.add("wed")
        elif p.startswith("thu"): days.add("thu")
        elif p.startswith("fri"): days.add("fri")
        elif p.startswith("sat"): days.add("sat")
        elif p.startswith("sun"): days.add("sun")
    return days

df["days_set"] = df[FREQ_COL].apply(parse_days_list)
df["crane_needed"] = df[CRANE_COL].apply(lambda x: int(truthy(x)))

# --------------------
# 2) 19–25 Aralık için günlük planı üret
# --------------------
dates = pd.date_range(DATE_START, DATE_END, freq="D")

rows = []
plan_dict = {}

for d in dates:
    day_en = d.day_name().lower()          # monday...
    day_key = DAY_MAP[day_en]              # mon...
    weekday_tr = TR_WEEKDAYS[day_en]

    # o gün toplanacak mahalleler: days_set içinde day_key geçenler
    target = df[df["days_set"].apply(lambda s: day_key in s)].copy()

    plan_dict[str(d.date())] = []

    for _, r in target.iterrows():
        neigh = str(r[NEIGH_COL]).strip()
        if not neigh or neigh.lower() == "nan":
            continue

        rows.append({
            "date": str(d.date()),
            "weekday_tr": weekday_tr,
            "Mahalle": neigh,
            "crane_needed": int(r["crane_needed"]),
            "garbage_truck_type": r.get("garbage_truck_type", None),
            "days_collected_per_week": r.get("days_collected_per_week", None),
            "collection_frequency": r.get(FREQ_COL, None)
        })

        plan_dict[str(d.date())].append({
            "Mahalle": neigh,
            "crane_needed": int(r["crane_needed"]),
            "garbage_truck_type": r.get("garbage_truck_type", None)
        })

service_plan = pd.DataFrame(rows)

if service_plan.empty:
    raise ValueError("service_plan yine boş geldi. frequency parsing başarısız. Örnek frequency değerlerini yazdırıp bakacağız.")

# --------------------
# 3) Özet + Kaydet
# --------------------
summary = service_plan.groupby(["date", "weekday_tr"], as_index=False).agg(
    neighborhoods=("Mahalle", "nunique"),
    crane_neighborhoods=("crane_needed", "sum")
)

service_plan.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
summary.to_csv(OUT_SUM, index=False, encoding="utf-8-sig")
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(plan_dict, f, ensure_ascii=False, indent=2)

print("\nFaz 2 tamam.")
print("Yazılan dosyalar:")
print(" -", OUT_CSV.resolve())
print(" -", OUT_JSON.resolve())
print(" -", OUT_SUM.resolve())
print("\nÖzet (ilk 7 satır):")
print(summary.head(7))

print("\nKontrol | frequency örnekleri:")
print(df[FREQ_COL].value_counts().head(5))


Faz2 | Kolonlar: ['mahalle_adi', 'garbage_truck_type', 'days_collected_per_week', 'collection_frequency_(truck_type)', 'is_crane_used', 'crane_rotation_days']
Faz2 | Satır sayısı: 68

Faz 2 tamam.
Yazılan dosyalar:
 - C:\Users\sefas\OneDrive\Masaüstü\NB_hackathon_2025-main\service_plan_by_date.csv
 - C:\Users\sefas\OneDrive\Masaüstü\NB_hackathon_2025-main\service_plan_by_date.json
 - C:\Users\sefas\OneDrive\Masaüstü\NB_hackathon_2025-main\service_plan_summary.csv

Özet (ilk 7 satır):
         date weekday_tr  neighborhoods  crane_neighborhoods
0  2025-12-19       Cuma             38                    5
1  2025-12-20  Cumartesi             31                    8
2  2025-12-21      Pazar              2                    0
3  2025-12-22  Pazartesi             38                    5
4  2025-12-23       Salı             24                    7
5  2025-12-24   Çarşamba             38                    5
6  2025-12-25   Perşembe             31                    8

Kontrol | frequency ör

In [4]:
service_plan_df = pd.read_csv("service_plan_summary.csv")
service_plan_df.head()

,date,weekday_tr,neighborhoods,crane_neighborhoods
0,2025-12-19,Cuma,38,5
1,2025-12-20,Cumartesi,31,8
2,2025-12-21,Pazar,2,0
3,2025-12-22,Pazartesi,38,5
4,2025-12-23,Salı,24,7


In [5]:
service_plan_date_df = pd.read_csv("service_plan_by_date.csv")
service_plan_date_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     211 non-null    object 
 1   weekday_tr               211 non-null    object 
 2   Mahalle                  211 non-null    object 
 3   crane_needed             211 non-null    int64  
 4   garbage_truck_type       211 non-null    object 
 5   days_collected_per_week  211 non-null    float64
 6   collection_frequency     211 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 11.7+ KB


In [6]:
sp = pd.read_csv("service_plan_by_date.csv")
sp[sp["date"]=="2025-12-21"][["Mahalle","crane_needed","garbage_truck_type","collection_frequency"]]


,Mahalle,crane_needed,garbage_truck_type,collection_frequency
72,DUMLUPINAR MAHALLESİ,0,Large Garbage Truck,"Monday , Thursday, Wednesday, Thursday, Friday..."
73,GÖRÜKLE MAHALLESİ,0,Large Garbage Truck,"Monday , Thursday, Wednesday, Thursday, Friday..."


In [7]:
sp.groupby(["crane_needed","garbage_truck_type"]).size().reset_index(name="count").sort_values("count", ascending=False)

,crane_needed,garbage_truck_type,count
0,0,Large Garbage Truck,154
2,1,Large Garbage Truck,38
1,0,Small Garbage Truck,19
